In [1]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd
import os
from dotenv import load_dotenv
import requests

load_dotenv()

github_token = os.getenv('GITHUB_TOKEN')

url = 'https://api.github.com/repos/petebrown/prepare-sql-db/contents/trfc.db'
headers = {
    'Authorization': f'token {github_token}',
    'Accept': 'application/vnd.github.v3.raw'
}

response = requests.get(url, headers=headers)

print(f"Response status: {response.status_code}")
print(f"Content length: {len(response.content)} bytes")

# Only write if we got a successful response
if response.status_code == 200:
    with open('trfc.db', 'wb') as f:
        f.write(response.content)
    print(f"File size after writing: {os.path.getsize('trfc.db')} bytes")
else:
    print(f"Error: {response.text}")

Response status: 200
Content length: 25698304 bytes
File size after writing: 25698304 bytes


In [2]:
db = database('trfc.db')

app, rt = fast_app()

In [18]:
seasons = ['2024/25', '2023/24']

placeholders = ','.join(['?' for _ in seasons])
[r['generic_comp'] for r in db.query(f"SELECT DISTINCT(generic_comp) FROM results r WHERE r.season IN ({placeholders}) ORDER BY generic_comp", seasons)]

["Associate Members' Cup", 'FA Cup', 'Football League', 'League Cup']

In [6]:
def get_plr_game_goals(season):

    params = [season]

    query = f"""
        SELECT 
            p.surname || ', ' || p.forename as player,
            r.game_no,
            COUNT(g.player_id) as goals_scored,
            COUNT(yc.player_id) as yellow_cards,
            COUNT(rc.player_id) as red_cards,
            COALESCE(rc.min_so, subs_off.sub_min_off, cgd.game_length, 90) - COALESCE(subs_on.sub_min_on, 0) as mins_played
        FROM results r
        LEFT JOIN player_apps pa ON r.game_date = pa.game_date
        LEFT JOIN players p ON pa.player_id = p.player_id
        LEFT JOIN goals g ON r.game_date = g.game_date AND pa.player_id = g.player_id
        LEFT JOIN cards_yellow yc ON r.game_date = yc.game_date AND pa.player_id = yc.player_id
        LEFT JOIN cards_red rc ON r.game_date = rc.game_date AND pa.player_id = rc.player_id
        LEFT JOIN subs_on ON r.game_date = subs_on.game_date AND pa.player_id = subs_on.player_on_id
        LEFT JOIN subs_off ON r.game_date = subs_off.game_date AND pa.player_id = subs_off.player_off_id
        LEFT JOIN cup_game_details cgd ON r.game_date = cgd.game_date
        WHERE r.season = ?
        GROUP BY player, r.game_no
        ORDER BY player, game_no
    """

    cursor = db.conn.execute(query, params)
    columns = [description[0] for description in cursor.description]
    return [dict(zip(columns, record)) for record in cursor.fetchall()]

app, rt = fast_app()

df = get_plr_game_goals('2002/03')

df = pd.DataFrame(df)

df['goals_scored'] = df['goals_scored'].astype('Int64')

goals_scored = df.pivot(
    index='player',
    columns='game_no',
    values='goals_scored'
)

mins_played = df.pivot(
    index='player',
    columns='game_no',
    values='mins_played'
)

def mins_cls(mins):
    if pd.isna(mins):
        return "mins-dnp"
    elif mins == 0:
        return "mins-inj-time"
    else:
        return f"mins-{int(mins)}"

@rt("/")
def get():
    return Titled(
        "Soccer Goals Table",
        P("Goals scored by player and game"),
        Table(
            Thead(
                Tr(
                    Th("Player"),
                    *[Th(col) for col in goals_scored.columns]
                )
            ),
            Tbody(
                *[Tr(
                    Td(player),
                    *[Td(str(val).replace('0', ''),
                        cls=mins_cls(mins_played.loc[player, col])
                      ) 
                      for val, col in zip(goals_scored.loc[player], goals_scored.columns)]
                ) for player in goals_scored.index]
            )
        )
    )

In [ ]:
server = JupyUvi(app)

In [ ]:
HTMX()

In [9]:
server.stop()